# Import libraries

In [1]:
from torchvision import transforms, datasets
import torchvision
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Choose and download dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import kagglehub

data = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")

print("Path to dataset files:", data)

Path to dataset files: /kaggle/input/cards-image-datasetclassification


# 2. Dataset preproccessing

In [4]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,) , (0.5,))
])

In [8]:
train_path = '/kaggle/input/cards-image-datasetclassification/train'
test_path = "/kaggle/input/cards-image-datasetclassification/test"

In [12]:
train = ImageFolder(train_path, transform=transform)
test = ImageFolder(test_path, transform=transform)
classes = train.classes
classes

['ace of clubs',
 'ace of diamonds',
 'ace of hearts',
 'ace of spades',
 'eight of clubs',
 'eight of diamonds',
 'eight of hearts',
 'eight of spades',
 'five of clubs',
 'five of diamonds',
 'five of hearts',
 'five of spades',
 'four of clubs',
 'four of diamonds',
 'four of hearts',
 'four of spades',
 'jack of clubs',
 'jack of diamonds',
 'jack of hearts',
 'jack of spades',
 'joker',
 'king of clubs',
 'king of diamonds',
 'king of hearts',
 'king of spades',
 'nine of clubs',
 'nine of diamonds',
 'nine of hearts',
 'nine of spades',
 'queen of clubs',
 'queen of diamonds',
 'queen of hearts',
 'queen of spades',
 'seven of clubs',
 'seven of diamonds',
 'seven of hearts',
 'seven of spades',
 'six of clubs',
 'six of diamonds',
 'six of hearts',
 'six of spades',
 'ten of clubs',
 'ten of diamonds',
 'ten of hearts',
 'ten of spades',
 'three of clubs',
 'three of diamonds',
 'three of hearts',
 'three of spades',
 'two of clubs',
 'two of diamonds',
 'two of hearts',
 'two o

In [13]:
train_loader = DataLoader(train, batch_size=32, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=False)
for img, label in train_loader:
    break
img.shape

torch.Size([32, 3, 128, 128])

# 3. Create Neural Network

In [15]:
class Model(nn.Module):
  def __init__(self, output_shape):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, stride=1),
        nn.BatchNorm2d(10),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=10, out_channels=30, kernel_size=3),
        nn.BatchNorm2d(30),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3),
        nn.BatchNorm2d(60),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer4 = nn.Sequential(
        nn.Conv2d(in_channels=60, out_channels=100, kernel_size=3),
        nn.BatchNorm2d(100),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Dropout(0.5),
        nn.Linear(in_features=3600, out_features=output_shape)
    )

  def forward(self, x):
    return self.classifier(self.layer4(self.layer3(self.layer2(self.layer1(x)))))

In [24]:
torch.manual_seed(42)
model = Model(len(classes)).to(device)

In [25]:
image_batch, label_batch = next(iter(train_loader))
image_batch = image_batch.to(device)
model(image_batch)

tensor([[-0.8565,  0.2473,  0.0395,  ...,  0.2567, -0.4270, -0.1690],
        [-1.5922,  0.9658,  0.4212,  ...,  2.3735, -0.5131,  1.4405],
        [ 0.9280,  0.3304,  2.0421,  ...,  2.1693, -2.8649, -1.2913],
        ...,
        [-0.6523,  0.4379,  1.3453,  ...,  0.4988, -1.2762,  1.0210],
        [-0.4801, -0.8914, -1.0964,  ..., -0.0391, -0.4271, -0.3809],
        [-0.8985,  0.1057,  2.0531,  ...,  1.4649,  0.5224,  0.3925]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [28]:
try :import torchinfo
except:
  !pip install torchinfo
  import torchinfo
from torchinfo import summary

summary(model , input_size=(1,3,128,128))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 53]                   --
├─Sequential: 1-1                        [1, 10, 63, 63]           --
│    └─Conv2d: 2-1                       [1, 10, 126, 126]         280
│    └─BatchNorm2d: 2-2                  [1, 10, 126, 126]         20
│    └─ReLU: 2-3                         [1, 10, 126, 126]         --
│    └─MaxPool2d: 2-4                    [1, 10, 63, 63]           --
├─Sequential: 1-2                        [1, 30, 30, 30]           --
│    └─Conv2d: 2-5                       [1, 30, 61, 61]           2,730
│    └─BatchNorm2d: 2-6                  [1, 30, 61, 61]           60
│    └─ReLU: 2-7                         [1, 30, 61, 61]           --
│    └─MaxPool2d: 2-8                    [1, 30, 30, 30]           --
├─Sequential: 1-3                        [1, 60, 14, 14]           --
│    └─Conv2d: 2-9                       [1, 60, 28, 28]           16,260
│    └─